Step 0: the original data files are copied locally in the following locations:

Ice data

In [1]:
ls -ltrh ./Data/ice

total 15608
-rw-rw-rw-  1 bogdan  staff   2.2M Oct 24 11:58 N_seaice_extent_daily_v3.0.csv
-rw-rw-rw-  1 bogdan  staff   2.2M Oct 24 12:03 S_seaice_extent_daily_v3.0.csv
-rw-rw-rw-@ 1 bogdan  staff   3.2M Oct 24 12:05 Sea_Ice_Index_Regional_Daily_Data_G02135_v3.0.xlsx


CO2 data

In [3]:
ls -ltrh ./Data/co2

total 168
-rw-r--r--@ 1 bogdan  staff    48K Nov 11 21:44 monthly_in_situ_co2.csv
-rw-r--r--@ 1 bogdan  staff    29K Nov 21 18:58 co2_mm_mlo.csv


Fire Data

In [4]:
ls -ltrh ./Data/fire

total 186833152
-rw-r--r--  1 bogdan  staff   1.4G Oct 24 14:56 fire_archive_M6_82356.json
-rw-r--r--  1 bogdan  staff    71M Oct 24 14:56 fire_nrt_M6_82356.json
-rw-r--r--  1 bogdan  staff   3.0G Oct 24 14:58 fire_archive_V1_82357.json
-rw-r--r--  1 bogdan  staff   1.6G Oct 24 15:01 fire_archive_M6_82363.json
-rw-r--r--  1 bogdan  staff   3.1G Oct 24 15:02 fire_nrt_V1_82357.json
-rw-r--r--  1 bogdan  staff   6.6G Oct 24 15:10 fire_archive_V1_82364.json
-rw-r--r--  1 bogdan  staff   1.7G Oct 25 10:27 fire_archive_M6_82479.json
-rw-r--r--  1 bogdan  staff   6.9G Oct 25 10:36 fire_archive_V1_82480.json
-rw-r--r--  1 bogdan  staff   1.6G Oct 25 10:37 fire_archive_M6_82481.json
-rw-r--r--  1 bogdan  staff   1.8G Oct 25 10:40 fire_archive_M6_82483.json
-rw-r--r--  1 bogdan  staff   7.4G Oct 25 10:49 fire_archive_V1_82484.json
-rw-r--r--  1 bogdan  staff   7.0G Oct 25 10:53 fire_archive_V1_82482.json
-rw-r--r--  1 bogdan  staff   1.6G Oct 25 10:54 fire_archive_M6_82487.json
-rw-r--r--  1 bog

FIRE

Step 1: convert fire files (JSON) to a JSON format that can be loaded with pd.read_json There are quotes that are missing in the original files. The following code was run for all the files, but I am showing how it runs for only one file.

In [5]:
import numpy as np
import pandas as pd
import random
import os
from scipy.signal import argrelextrema
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

The following code was run for all the files, but I am showing how it runs for only one file

In [6]:
fire_files = ['fire_archive_V1_87837.json']
#fire_files = ['fire_archive_M6_82356.json', 'fire_archive_M6_82363.json', 'fire_archive_M6_82479.json',
#'fire_archive_M6_82481.json', 'fire_archive_M6_82483.json', 'fire_archive_M6_82487.json',
#'fire_archive_M6_82489.json', 'fire_archive_M6_82491.json', 'fire_archive_M6_82493.json',
#'fire_archive_M6_82495.json', 'fire_archive_M6_82497.json', 'fire_archive_M6_82499.json',
#'fire_archive_M6_82501.json', 'fire_archive_M6_82503.json', 'fire_archive_M6_82505.json',
#'fire_archive_M6_82508.json', 'fire_archive_M6_82511.json', 'fire_archive_M6_82513.json',
#'fire_archive_M6_82515.json', 'fire_archive_M6_82517.json', 'fire_archive_V1_82357.json',
#'fire_archive_V1_82364.json', 'fire_archive_V1_82480.json', 'fire_archive_V1_82482.json',
#'fire_archive_V1_82484.json', 'fire_archive_V1_82488.json', 'fire_archive_V1_82490.json',
#'fire_nrt_M6_82356.json', 'fire_nrt_V1_82357.json']
strings = ['"acq_date": ','"acq_time": ','"acq_time": ','"instrument": ',
           '"satellite": ','"version": ','"daynight": ', '"confidence": ']
drops=[]
sample_percentage=0.01
print_progress=100000 # Print '*' after 100000 read observations (about 1000 written observations)

In [7]:
def add_quotation_mark(line, strings, drops):

    for string in drops:
        pos=line.find(string)
        if pos != -1:
            return None

    for string in strings:
        pos=line.find(string)
        if pos != -1:
            index = pos+len(string)
            if line[-2]==',':
                return line[:index]+'"'+line[index:-2]+'"'+',\n'
            else:
                return line[:index]+'"'+line[index:-1]+'"'+'\n' # no comma if it's the last line

    return line

In [8]:
def decision(probability):
    return random.random() < probability # return True/False depending on the the probability

In [9]:
def read_skip_record(input_file, output_file):
    global comma
    observation = ''
    while True:
        json_line = input_file.readline().strip() + '\n'
        processed_line = add_quotation_mark(json_line, strings, drops)
        observation += processed_line
        if processed_line[0] == '}':
            if(decision(sample_percentage)): # write a random "sample_percentage" of the records
                if processed_line[-2] == ',': # last char is '\n'
                    comma = True
                else:
                    comma = False
                output_file.write(observation)
            if processed_line[-2] == ',': # check for comma, last char in line is '\n'
                return True
            else:
                return False

In [10]:
for fire_file in fire_files:
    i = 0
    comma = False
    fire_file = 'Data/fire/' + fire_file
    pos=fire_file.find('.')
    fire_file_out = fire_file[:pos] + '_out' + fire_file[pos:]
    print('\n', fire_file, ' -> ', fire_file_out)

    with open(fire_file, 'r') as file:
        while True:
            with open(fire_file_out, 'w') as file1:
                line = file.readline().strip()
                # '[' on the first line
                if line != '[':
                    print('Wrong json structure!')
                    break
                else:
                    file1.write('['+'\n')
                while read_skip_record(file, file1):
                    i += 1
                    if not i%print_progress:
                        print("*", end="")
                # ']' on the last line
                file1.write(']'+'\n')
                break

    # if there is a comma at the end, erase it
    if comma: # delete comma after the last observation
        file1.close()
        with open(fire_file_out, 'rb+') as file1:
            file1.seek(-4, os.SEEK_END)
            file1.truncate()
            file1.close()
        with open(fire_file_out, 'a') as file1:
            file1.write('\n]'+'\n')


 Data/fire/fire_archive_V1_87837.json  ->  Data/fire/fire_archive_V1_87837_out.json
********************************************************************************************************************************************************************************************************

In [11]:
ls -ltrh ./Data/fire/fire_archive_V1_87837*

-rw-r--r--@ 1 bogdan  staff   7.0G Nov 20 08:04 ./Data/fire/fire_archive_V1_87837.json
-rw-r--r--  1 bogdan  staff    51M Nov 20 21:13 ./Data/fire/fire_archive_V1_87837_out_old.json
-rw-r--r--  1 bogdan  staff    51M Nov 23 14:55 ./Data/fire/fire_archive_V1_87837_out.json


Fire Data - preprocessed

In [12]:
fire_files_1=['fire_archive_M6_82517_out.json', 'fire_archive_M6_82515_out.json', 'fire_archive_M6_82513_out.json',
              'fire_archive_M6_82511_out.json', 'fire_archive_M6_82508_out.json', 'fire_archive_M6_82505_out.json',
              'fire_archive_M6_82503_out.json', 'fire_archive_M6_82501_out.json', 'fire_archive_M6_82497_out.json', 
              'fire_archive_M6_82495_out.json', 'fire_archive_M6_82493_out.json', 'fire_archive_M6_82491_out.json',
              'fire_archive_M6_82489_out.json', 'fire_archive_M6_82487_out.json', 'fire_archive_M6_87748_out.json',
              'fire_archive_M6_82483_out.json', 'fire_archive_M6_82481_out.json', 'fire_archive_M6_82479_out.json',
              'fire_archive_M6_82363_out.json', 'fire_archive_M6_82356_out.json', 'fire_nrt_M6_82356_out.json']

fire_files_2=['fire_archive_V1_82490_out.json', 'fire_archive_V1_82488_out.json', 'fire_archive_V1_87837_out.json',
              'fire_archive_V1_82484_out.json', 'fire_archive_V1_82482_out.json', 'fire_archive_V1_82480_out.json',
              'fire_archive_V1_82364_out.json', 'fire_archive_V1_82357_out.json', 'fire_nrt_V1_82357_out.json']

In [13]:
fire1_df_list = []
for item in fire_files_1:
    print('Loading: ', item, '...')
    fire1_df_list.append(pd.read_json('Data/fire/' + item))

Loading:  fire_archive_M6_82517_out.json ...
Loading:  fire_archive_M6_82515_out.json ...
Loading:  fire_archive_M6_82513_out.json ...
Loading:  fire_archive_M6_82511_out.json ...
Loading:  fire_archive_M6_82508_out.json ...
Loading:  fire_archive_M6_82505_out.json ...
Loading:  fire_archive_M6_82503_out.json ...
Loading:  fire_archive_M6_82501_out.json ...
Loading:  fire_archive_M6_82497_out.json ...
Loading:  fire_archive_M6_82495_out.json ...
Loading:  fire_archive_M6_82493_out.json ...
Loading:  fire_archive_M6_82491_out.json ...
Loading:  fire_archive_M6_82489_out.json ...
Loading:  fire_archive_M6_82487_out.json ...
Loading:  fire_archive_M6_87748_out.json ...
Loading:  fire_archive_M6_82483_out.json ...
Loading:  fire_archive_M6_82481_out.json ...
Loading:  fire_archive_M6_82479_out.json ...
Loading:  fire_archive_M6_82363_out.json ...
Loading:  fire_archive_M6_82356_out.json ...
Loading:  fire_nrt_M6_82356_out.json ...


In [14]:
fire1_df_list[0].columns

Index(['acq_date', 'acq_time', 'bright_t31', 'brightness', 'confidence',
       'daynight', 'frp', 'instrument', 'latitude', 'longitude', 'satellite',
       'scan', 'track', 'type', 'version'],
      dtype='object')

In [15]:
for i in range(len(fire1_df_list)):
    print(i, min(fire1_df_list[i]['acq_date']), max(fire1_df_list[i]['acq_date']), 
          len(fire1_df_list[i]['acq_date'].unique()))

0 2000-11-01 2000-12-31 61
1 2001-01-01 2001-12-31 348
2 2002-01-01 2002-12-31 355
3 2003-01-01 2003-12-31 365
4 2004-01-01 2004-12-31 366
5 2005-01-01 2005-12-31 365
6 2006-01-01 2006-12-31 365
7 2007-01-01 2007-12-31 365
8 2008-01-01 2008-12-31 366
9 2009-01-01 2009-12-31 365
10 2010-01-01 2010-12-31 365
11 2011-01-01 2011-12-31 365
12 2012-01-31 2012-12-31 336
13 2013-01-01 2013-12-31 365
14 2014-01-01 2014-12-31 365
15 2015-01-01 2015-12-31 365
16 2016-01-01 2016-12-31 366
17 2017-01-01 2017-12-31 365
18 2018-01-01 2018-12-31 365
19 2019-01-01 2019-09-30 273
20 2019-10-01 2019-10-24 24


Notes:

there are some missing values for old data (years 2000, 2001, 2002)

file 19 and file 20 refer to the same year 2019
The 2019 file fire_nrt_M6_82356_out.json represents near real time data and
the observations from these two files will merge correctly for the year 2019 until October 24
because the files were downloaded on October 25

In [16]:
fire_M6 = pd.DataFrame()
for i in range(len(fire1_df_list)):
    print('Concatenating '+ fire_files_1[i] + '...')
    fire_M6 = pd.concat([fire_M6, fire1_df_list[i]], sort=False)

Concatenating fire_archive_M6_82517_out.json...
Concatenating fire_archive_M6_82515_out.json...
Concatenating fire_archive_M6_82513_out.json...
Concatenating fire_archive_M6_82511_out.json...
Concatenating fire_archive_M6_82508_out.json...
Concatenating fire_archive_M6_82505_out.json...
Concatenating fire_archive_M6_82503_out.json...
Concatenating fire_archive_M6_82501_out.json...
Concatenating fire_archive_M6_82497_out.json...
Concatenating fire_archive_M6_82495_out.json...
Concatenating fire_archive_M6_82493_out.json...
Concatenating fire_archive_M6_82491_out.json...
Concatenating fire_archive_M6_82489_out.json...
Concatenating fire_archive_M6_82487_out.json...
Concatenating fire_archive_M6_87748_out.json...
Concatenating fire_archive_M6_82483_out.json...
Concatenating fire_archive_M6_82481_out.json...
Concatenating fire_archive_M6_82479_out.json...
Concatenating fire_archive_M6_82363_out.json...
Concatenating fire_archive_M6_82356_out.json...
Concatenating fire_nrt_M6_82356_out.json

In [17]:
type(fire_M6)

pandas.core.frame.DataFrame

In [18]:
fire_M6.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 849308 entries, 0 to 1947
Data columns (total 15 columns):
acq_date      849308 non-null object
acq_time      849308 non-null int64
bright_t31    849308 non-null float64
brightness    849308 non-null float64
confidence    849308 non-null int64
daynight      849308 non-null object
frp           849308 non-null float64
instrument    849308 non-null object
latitude      849308 non-null float64
longitude     849308 non-null float64
satellite     849308 non-null object
scan          849308 non-null float64
track         849308 non-null float64
type          847360 non-null float64
version       849308 non-null object
dtypes: float64(8), int64(2), object(5)
memory usage: 103.7+ MB


In [19]:
# There are no observations before 2000-11-01 and after 2019-10-24
# There are 1939 missing values for type column
print('Min date: ', min(fire_M6['acq_date']), '   Max date: ', max(fire_M6['acq_date']))

Min date:  2000-11-01    Max date:  2019-10-24


In [20]:
fire_M6.set_index('acq_date')

,acq_time,bright_t31,brightness,confidence,daynight,frp,instrument,latitude,longitude,satellite,scan,track,type,version
acq_date,,,,,,,,,,,,,,
2000-11-01,17,309.3,324.7,35,D,26.8,MODIS,-20.8650,146.3039,Terra,2.3,1.5,0.0,6.2
2000-11-01,17,308.2,331.7,81,D,68.8,MODIS,-20.5602,146.0194,Terra,2.4,1.5,0.0,6.2
2000-11-01,18,302.8,319.5,62,D,7.1,MODIS,-24.3197,151.5619,Terra,1.0,1.0,0.0,6.2
2000-11-01,155,319.5,334.9,59,D,10.5,MODIS,-18.0912,128.9856,Terra,1.0,1.0,0.0,6.2
2000-11-01,157,317.6,367.9,100,D,100.3,MODIS,-25.5102,125.4199,Terra,1.1,1.0,0.0,6.2
2000-11-01,157,318.7,339.8,80,D,26.6,MODIS,-25.2563,125.0587,Terra,1.1,1.1,0.0,6.2
2000-11-01,158,315.8,335.0,24,D,61.6,MODIS,-27.0504,119.5368,Terra,2.3,1.5,0.0,6.2
2000-11-01,315,284.1,358.9,99,D,105.1,MODIS,50.2959,125.5208,Terra,1.1,1.1,0.0,6.2
2000-11-01,315,279.4,307.8,66,D,46.7,MODIS,48.6393,133.4861,Terra,2.9,1.6,0.0,6.2


In [21]:
ls -ltrh ./Data/fire/*V1*

-rw-r--r--  1 bogdan  staff   3.0G Oct 24 14:58 ./Data/fire/fire_archive_V1_82357.json
-rw-r--r--  1 bogdan  staff   3.1G Oct 24 15:02 ./Data/fire/fire_nrt_V1_82357.json
-rw-r--r--  1 bogdan  staff   6.6G Oct 24 15:10 ./Data/fire/fire_archive_V1_82364.json
-rw-r--r--  1 bogdan  staff   6.9G Oct 25 10:36 ./Data/fire/fire_archive_V1_82480.json
-rw-r--r--  1 bogdan  staff   7.4G Oct 25 10:49 ./Data/fire/fire_archive_V1_82484.json
-rw-r--r--  1 bogdan  staff   7.0G Oct 25 10:53 ./Data/fire/fire_archive_V1_82482.json
-rw-r--r--  1 bogdan  staff   6.7G Oct 25 11:13 ./Data/fire/fire_archive_V1_82488.json
-rw-r--r--  1 bogdan  staff   7.0G Oct 25 11:19 ./Data/fire/fire_archive_V1_82490.json
-rw-r--r--  1 bogdan  staff    22M Nov  5 20:39 ./Data/fire/fire_archive_V1_82357_out.json
-rw-r--r--  1 bogdan  staff    48M Nov  5 20:52 ./Data/fire/fire_archive_V1_82364_out.json
-rw-r--r--  1 bogdan  staff    50M Nov  5 21:05 ./Data/fire/fire_archive_V1_82480_out.json
-rw-r--r--  1 bogdan  staff    51M 

In [22]:
fire2_df_list = []
for item in fire_files_2:
    print('Loading: ', item, '...')
    fire2_df_list.append(pd.read_json('Data/fire/' + item))

Loading:  fire_archive_V1_82490_out.json ...
Loading:  fire_archive_V1_82488_out.json ...
Loading:  fire_archive_V1_87837_out.json ...
Loading:  fire_archive_V1_82484_out.json ...
Loading:  fire_archive_V1_82482_out.json ...
Loading:  fire_archive_V1_82480_out.json ...
Loading:  fire_archive_V1_82364_out.json ...
Loading:  fire_archive_V1_82357_out.json ...
Loading:  fire_nrt_V1_82357_out.json ...


In [23]:
for i in range(len(fire2_df_list)):
    print(i, min(fire2_df_list[i]['acq_date']), max(fire2_df_list[i]['acq_date']), 
          len(fire2_df_list[i]['acq_date'].unique()))

0 2012-01-31 2012-12-31 335
1 2013-01-01 2013-12-31 365
2 2014-01-01 2014-12-31 364
3 2015-01-01 2015-12-31 365
4 2016-01-01 2016-12-31 366
5 2017-01-01 2017-12-31 365
6 2018-01-01 2018-12-31 365
7 2019-01-01 2019-06-30 181
8 2019-07-01 2019-10-24 116


Notes:

There are some missing values for old data (years 2012, 2014)

file 7 and file 8 refer to the same year 2019
The 2019 file fire_nrt_V1_82357_out.json represents near real time data and
the observations from these two files will merge correctly for the year 2019 until October 24
because the files were downloaded on October 25
There are NaN values at "daynight" column

In [24]:
fire_V1 = pd.DataFrame()
for i in range(len(fire2_df_list)):
    print('Concatenating '+ fire_files_2[i] + '...')
    fire_V1 = pd.concat([fire_V1, fire2_df_list[i]], sort=False)

Concatenating fire_archive_V1_82490_out.json...
Concatenating fire_archive_V1_82488_out.json...
Concatenating fire_archive_V1_87837_out.json...
Concatenating fire_archive_V1_82484_out.json...
Concatenating fire_archive_V1_82482_out.json...
Concatenating fire_archive_V1_82480_out.json...
Concatenating fire_archive_V1_82364_out.json...
Concatenating fire_archive_V1_82357_out.json...
Concatenating fire_nrt_V1_82357_out.json...


In [25]:
type(fire_V1)

pandas.core.frame.DataFrame

In [26]:
fire_V1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1580506 entries, 0 to 88072
Data columns (total 15 columns):
acq_date      1580506 non-null object
acq_time      1580506 non-null int64
bright_ti4    1580506 non-null float64
bright_ti5    1580506 non-null float64
confidence    1580506 non-null object
frp           1580506 non-null float64
instrument    1580506 non-null object
latitude      1580506 non-null float64
longitude     1580506 non-null float64
satellite     1580506 non-null object
scan          1580506 non-null float64
track         1580506 non-null float64
type          1492433 non-null float64
version       1580506 non-null object
daynight      88073 non-null object
dtypes: float64(8), int64(1), object(6)
memory usage: 192.9+ MB


In [27]:
# There are no observations before 2012-01-31 and after 2019-10-24
print('Min date: ', min(fire_V1['acq_date']), '   Max date: ', max(fire_V1['acq_date']))

Min date:  2012-01-31    Max date:  2019-10-24


In [28]:
fire_V1.set_index('acq_date')

,acq_time,bright_ti4,bright_ti5,confidence,frp,instrument,latitude,longitude,satellite,scan,track,type,version,daynight
acq_date,,,,,,,,,,,,,,
2012-01-31,104,324.2,263.0,n,3.2,VIIRS,50.42116,18.13946,N,0.41,0.37,2.0,1,NaN
2012-01-31,105,337.6,266.2,n,8.0,VIIRS,49.81459,18.31155,N,0.41,0.37,2.0,1,NaN
2012-01-31,110,300.6,275.6,n,5.2,VIIRS,31.65900,6.06138,N,0.43,0.38,2.0,1,NaN
2012-01-31,110,297.0,271.1,n,1.4,VIIRS,32.92940,3.24130,N,0.39,0.44,2.0,1,NaN
2012-01-31,110,349.4,279.6,n,4.3,VIIRS,29.70964,19.71412,N,0.40,0.60,2.0,1,NaN
2012-01-31,111,331.7,275.1,n,3.1,VIIRS,28.64950,7.68689,N,0.38,0.36,2.0,1,NaN
2012-01-31,115,304.1,292.8,n,0.9,VIIRS,11.81456,4.97983,N,0.39,0.36,0.0,1,NaN
2012-01-31,115,310.9,293.6,n,1.3,VIIRS,10.39675,13.45527,N,0.40,0.60,0.0,1,NaN
2012-01-31,115,317.3,286.3,n,1.9,VIIRS,13.73985,13.83765,N,0.37,0.58,0.0,1,NaN


In [29]:
scan_M6 = fire_M6['scan'].unique()
scan_V1 = fire_V1['scan'].unique()

In [30]:
print(np.sort(scan_M6), '\n')
print(np.sort(scan_V1))

[1.  1.1 1.2 1.3 1.4 1.5 1.6 1.7 1.8 1.9 2.  2.1 2.2 2.3 2.4 2.5 2.6 2.7
 2.8 2.9 3.  3.1 3.2 3.3 3.4 3.5 3.6 3.7 3.8 3.9 4.  4.1 4.2 4.3 4.4 4.5
 4.6 4.7 4.8] 

[0.32 0.33 0.34 0.35 0.36 0.37 0.38 0.39 0.4  0.41 0.42 0.43 0.44 0.45
 0.46 0.47 0.48 0.49 0.5  0.51 0.52 0.53 0.54 0.55 0.56 0.57 0.58 0.59
 0.6  0.61 0.62 0.63 0.64 0.65 0.66 0.67 0.68 0.69 0.7  0.71 0.72 0.73
 0.74 0.75 0.76 0.77 0.78 0.79 0.8 ]


In [31]:
track_M6 = fire_M6['track'].unique()
track_V1 = fire_V1['track'].unique()

In [32]:
print(np.sort(track_M6), '\n')
print(np.sort(track_V1))

[1.  1.1 1.2 1.3 1.4 1.5 1.6 1.7 1.8 1.9 2. ] 

[0.36 0.37 0.38 0.39 0.4  0.41 0.42 0.43 0.44 0.45 0.46 0.47 0.48 0.49
 0.5  0.51 0.52 0.53 0.54 0.55 0.56 0.57 0.58 0.59 0.6  0.61 0.62 0.63
 0.64 0.65 0.66 0.67 0.68 0.69 0.7  0.71 0.72 0.73 0.74 0.75 0.76 0.77
 0.78]


Columns 'scan' and 'track' are useful for calculating the size of the pixel. At nadir, it is 1km for M6 dataframe and 375m for V1 dataframe. For scans and tracks that are close to the minimum and maximum values, the observed pixel size increases because the observation angle decreases. However, we are not concerned about the resolution. The radiative power per unit of surface is more important and this value takes into account ther 'track' and 'scan' values.
Therefore, it is safe to drop these two columns.

In [130]:
fire_M6.drop(['scan', 'track'], axis=1, inplace=True)
fire_V1.drop(['scan', 'track'], axis=1, inplace=True)

In [33]:
# There are only three satellites that gather data: Terra, Aqua and Suomi NPP (coded 'N' in fire_V1 dataset)
print(fire_M6['satellite'].unique())
print(fire_V1['satellite'].unique())

['Terra' 'Aqua']
['N']


In [34]:
# Since the name is not important, this column can be dropped
fire_M6.drop('satellite', axis=1, inplace=True)
fire_V1.drop('satellite', axis=1, inplace=True)

In [37]:
# There are missing values for "type" in both dataframes
# Type column has the following meaning:
# type 0: presumed vegetation fire
# type 1: active volcano
# type 2: other static land source
# type 3: offshore
# Since we are interested only in vegetation fire, we can use only the subset where type=0 and drop the column

In [38]:
fire_M6.count()

acq_date      849308
acq_time      849308
bright_t31    849308
brightness    849308
confidence    849308
daynight      849308
frp           849308
instrument    849308
latitude      849308
longitude     849308
scan          849308
track         849308
type          847360
version       849308
dtype: int64

In [39]:
fire_M6[fire_M6['type']==0.0].count()

acq_date      817400
acq_time      817400
bright_t31    817400
brightness    817400
confidence    817400
daynight      817400
frp           817400
instrument    817400
latitude      817400
longitude     817400
scan          817400
track         817400
type          817400
version       817400
dtype: int64

In [40]:
fire_M6 = fire_M6[fire_M6['type']==0.0]

In [41]:
fire_V1.count()

acq_date      1580506
acq_time      1580506
bright_ti4    1580506
bright_ti5    1580506
confidence    1580506
frp           1580506
instrument    1580506
latitude      1580506
longitude     1580506
scan          1580506
track         1580506
type          1492433
version       1580506
daynight        88073
dtype: int64

In [42]:
fire_V1[fire_V1['type']==0.0].count()

acq_date      1353795
acq_time      1353795
bright_ti4    1353795
bright_ti5    1353795
confidence    1353795
frp           1353795
instrument    1353795
latitude      1353795
longitude     1353795
scan          1353795
track         1353795
type          1353795
version       1353795
daynight            0
dtype: int64

In [43]:
fire_V1 = fire_V1[fire_V1['type']==0.0]

In [44]:
fire_M6.drop('type', axis=1, inplace=True)
fire_V1.drop('type', axis=1, inplace=True)

In [45]:
# "version" column represents the source of data and will be dropped

In [46]:
version_M6 = fire_M6['version'].unique()
version_V1 = fire_V1['version'].unique()

In [47]:
print(version_M6)
print(version_V1)

[6.2 6.3]
[1]


In [48]:
fire_M6.drop('version', axis=1, inplace=True)
fire_V1.drop('version', axis=1, inplace=True)

In [49]:
# There are two types of instruments that measure the same parameters (at different resolutions)
# This column can be dropped

In [50]:
instrument_M6 = fire_M6['instrument'].unique()
instrument_V1 = fire_V1['instrument'].unique()

In [51]:
print(instrument_M6)
print(instrument_V1)

['MODIS']
['VIIRS']


In [52]:
fire_M6.drop('instrument', axis=1, inplace=True)
fire_V1.drop('instrument', axis=1, inplace=True)

In [53]:
fire_M6.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 817400 entries, 0 to 36736
Data columns (total 11 columns):
acq_date      817400 non-null object
acq_time      817400 non-null int64
bright_t31    817400 non-null float64
brightness    817400 non-null float64
confidence    817400 non-null int64
daynight      817400 non-null object
frp           817400 non-null float64
latitude      817400 non-null float64
longitude     817400 non-null float64
scan          817400 non-null float64
track         817400 non-null float64
dtypes: float64(7), int64(2), object(2)
memory usage: 74.8+ MB


In [54]:
fire_V1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1353795 entries, 6 to 87314
Data columns (total 11 columns):
acq_date      1353795 non-null object
acq_time      1353795 non-null int64
bright_ti4    1353795 non-null float64
bright_ti5    1353795 non-null float64
confidence    1353795 non-null object
frp           1353795 non-null float64
latitude      1353795 non-null float64
longitude     1353795 non-null float64
scan          1353795 non-null float64
track         1353795 non-null float64
daynight      0 non-null object
dtypes: float64(7), int64(1), object(3)
memory usage: 123.9+ MB


In [55]:
# "daynight" column can be calculated from "longitude", "latitude", "acq_date" and "acq_time"
# Therefore it is a feature that is lineary dependent of other features and must be dropped

In [56]:
fire_M6.drop('daynight', axis=1, inplace=True)
fire_V1.drop('daynight', axis=1, inplace=True)

In [57]:
# acq_type has a numpy.int64 format with the first 2 digits for hours and the last 2 digits for minutes

In [58]:
fire_V1['acq_time'][:20]

6     115
7     115
8     115
9     115
10    116
11    116
12    116
13    116
14    116
15    116
16    116
17    116
18    116
19    116
20    116
21    116
23    117
24    117
25    117
26    256
Name: acq_time, dtype: int64

In [59]:
type(fire_V1['acq_time'].iloc[0])

numpy.int64

This format will be changed to a float number with 5 decimals. The new columns is named "decimal_date"

In [60]:
def decimal_time(date):
    return np.round((int(date/100)*60 + date%100)/1440, 5)

In [61]:
np.round(1439/1440, 5)

0.99931

In [62]:
fire_M6['decimal_time'] = list(map(lambda x: decimal_time(x), fire_M6['acq_time']))
fire_V1['decimal_time'] = list(map(lambda x: decimal_time(x), fire_V1['acq_time']))

In [63]:
print(decimal_time(0), decimal_time(2359))
print(fire_M6['decimal_time'].min(), fire_M6['decimal_time'].max())
print(fire_V1['decimal_time'].min(), fire_V1['decimal_time'].max())

0.0 0.99931
0.0 0.99931
0.0 0.99931


In [64]:
fire_M6.drop('acq_time', axis=1, inplace=True)
fire_V1.drop('acq_time', axis=1, inplace=True)

In [65]:
fire_M6.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 817400 entries, 0 to 36736
Data columns (total 10 columns):
acq_date        817400 non-null object
bright_t31      817400 non-null float64
brightness      817400 non-null float64
confidence      817400 non-null int64
frp             817400 non-null float64
latitude        817400 non-null float64
longitude       817400 non-null float64
scan            817400 non-null float64
track           817400 non-null float64
decimal_time    817400 non-null float64
dtypes: float64(8), int64(1), object(1)
memory usage: 68.6+ MB


In [66]:
fire_V1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1353795 entries, 6 to 87314
Data columns (total 10 columns):
acq_date        1353795 non-null object
bright_ti4      1353795 non-null float64
bright_ti5      1353795 non-null float64
confidence      1353795 non-null object
frp             1353795 non-null float64
latitude        1353795 non-null float64
longitude       1353795 non-null float64
scan            1353795 non-null float64
track           1353795 non-null float64
decimal_time    1353795 non-null float64
dtypes: float64(8), object(2)
memory usage: 113.6+ MB


In both datasets, the frp column represents the Fire Radiative Power. According to the scientific paper https://agupubs.onlinelibrary.wiley.com/doi/10.1029/2009JD013769, frp is measured in MW and it depends of of the brightness of the pixel and of adjacent pixels, after substracting the radiation emitted by surrounding areas that are not influenced by the fire.

This column "frp" is the most important measurement for the fire datasets.
The brightness is useful for estimating the temperature in degrees K. Although frp is the most useful column, brightness can add value. A small area of burning trees can have a higher brightness than a larger area of burning grass - but may have a similar frp.

Brightness measurements come from different channels: 21/22 and 31 for fire_M6 and 1...4 and I5 for fire_V1. in order to keep things simple, I shall create a column "mean_brightness" with the mean value from different channels, for both datasets.

In [67]:
fire_M6.describe()

,bright_t31,brightness,confidence,frp,latitude,longitude,scan,track,decimal_time
count,817400.000000,817400.000000,817400.000000,817400.000000,817400.000000,817400.000000,817400.000000,817400.000000,817400.000000
mean,300.774716,326.649117,67.030804,44.000654,5.033748,22.666590,1.605078,1.210288,0.470902
std,8.775355,17.338682,20.514848,100.452129,23.658475,63.130978,0.802800,0.245667,0.200293
min,264.000000,300.000000,0.000000,0.000000,-53.666100,-175.074900,1.000000,1.000000,0.000000
25%,295.300000,316.200000,54.000000,11.500000,-12.326400,-7.848325,1.100000,1.000000,0.344440
50%,301.300000,323.800000,69.000000,20.800000,-0.911250,24.369950,1.300000,1.100000,0.488890
75%,306.500000,332.900000,82.000000,42.500000,13.401100,44.395800,1.800000,1.300000,0.577780
max,400.100000,506.500000,100.000000,11777.000000,73.012900,179.671900,4.800000,2.000000,0.999310


In [68]:
fire_V1.describe()

,bright_ti4,bright_ti5,frp,latitude,longitude,scan,track,decimal_time
count,1.353795e+06,1.353795e+06,1.353795e+06,1.353795e+06,1.353795e+06,1.353795e+06,1.353795e+06,1.353795e+06
mean,3.364999e+02,2.998030e+02,1.042217e+01,4.671090e+00,2.796286e+01,4.596634e-01,4.809523e-01,5.031079e-01
std,1.678221e+01,1.062362e+01,2.295652e+01,2.248233e+01,6.215099e+01,8.705421e-02,1.143738e-01,2.122116e-01
min,2.080000e+02,0.000000e+00,0.000000e+00,-8.746420e+01,-1.789793e+02,3.200000e-01,3.600000e-01,0.000000e+00
25%,3.313000e+02,2.929000e+02,3.000000e+00,-1.218388e+01,5.385705e+00,3.900000e-01,3.800000e-01,3.993100e-01
50%,3.381000e+02,3.007000e+02,5.400000e+00,2.408370e+00,2.627994e+01,4.400000e-01,4.500000e-01,4.958300e-01
75%,3.460000e+02,3.066000e+02,1.010000e+01,1.322147e+01,6.483222e+01,5.100000e-01,5.600000e-01,6.048600e-01
max,3.670000e+02,3.800000e+02,7.502900e+03,7.265458e+01,1.798701e+02,8.000000e-01,7.800000e-01,9.993100e-01


In [69]:
fire_M6['mean_brightness'] = list(round((fire_M6['brightness']+fire_M6['bright_t31'])/2, 3))

In [70]:
fire_M6.drop(['brightness', 'bright_t31'], axis=1, inplace=True)

In [71]:
fire_M6.describe()

,confidence,frp,latitude,longitude,scan,track,decimal_time,mean_brightness
count,817400.000000,817400.000000,817400.000000,817400.000000,817400.000000,817400.000000,817400.000000,817400.000000
mean,67.030804,44.000654,5.033748,22.666590,1.605078,1.210288,0.470902,313.711916
std,20.514848,100.452129,23.658475,63.130978,0.802800,0.245667,0.200293,11.596467
min,0.000000,0.000000,-53.666100,-175.074900,1.000000,1.000000,0.000000,282.600000
25%,54.000000,11.500000,-12.326400,-7.848325,1.100000,1.000000,0.344440,306.550000
50%,69.000000,20.800000,-0.911250,24.369950,1.300000,1.100000,0.488890,312.900000
75%,82.000000,42.500000,13.401100,44.395800,1.800000,1.300000,0.577780,319.450000
max,100.000000,11777.000000,73.012900,179.671900,4.800000,2.000000,0.999310,453.300000


In [72]:
fire_V1['mean_brightness'] = list(round((fire_V1['bright_ti5']+fire_V1['bright_ti4'])/2, 3))

In [73]:
fire_V1.drop(['bright_ti5', 'bright_ti4'], axis=1, inplace=True)

In [74]:
fire_V1.describe()

,frp,latitude,longitude,scan,track,decimal_time,mean_brightness
count,1.353795e+06,1.353795e+06,1.353795e+06,1.353795e+06,1.353795e+06,1.353795e+06,1.353795e+06
mean,1.042217e+01,4.671090e+00,2.796286e+01,4.596634e-01,4.809523e-01,5.031079e-01,3.181515e+02
std,2.295652e+01,2.248233e+01,6.215099e+01,8.705421e-02,1.143738e-01,2.122116e-01,1.224142e+01
min,0.000000e+00,-8.746420e+01,-1.789793e+02,3.200000e-01,3.600000e-01,0.000000e+00,1.605000e+02
25%,3.000000e+00,-1.218388e+01,5.385705e+00,3.900000e-01,3.800000e-01,3.993100e-01,3.132500e+02
50%,5.400000e+00,2.408370e+00,2.627994e+01,4.400000e-01,4.500000e-01,4.958300e-01,3.198500e+02
75%,1.010000e+01,1.322147e+01,6.483222e+01,5.100000e-01,5.600000e-01,6.048600e-01,3.257000e+02
max,7.502900e+03,7.265458e+01,1.798701e+02,8.000000e-01,7.800000e-01,9.993100e-01,3.735000e+02


In [75]:
fire_M6['confidence'].unique()

array([ 35,  81,  62,  59, 100,  80,  24,  99,  66,  60,  91,  97,  78,
        77,  67,  93,   0,  63,  28,  41,  79,  69,  57,  70,  36,  75,
        86,  88,  52,  85,  61,  58,  64,  73,  90,  37,  18,  68,  65,
        83,  94,  44,  55,  49,  76,  39,  92,  53,  29,  25,  95,  51,
        84,  38,  45,  74,  48,  50,  54,  71,  89,  82,  23,  87,  40,
        46,  16,  56,  43,  42,  14,  33,  11,  72,  96,  20,  27,  47,
        34,  26,  31,  98,  30,  13,  32,  21,  17,  19,   5,  22,   8,
         3,   2,  12,  15,   4,   6,   9,  10,   1,   7])

In [76]:
fire_V1['confidence'].unique()

array(['n', 'h', 'l'], dtype=object)

The final issue to be handled is the confidence column. In fire_M6 dataset it is measured between 0 and 100 (step 1) while in fire_V1 dataset it is a qualitative value: "n" = normal, "l" = low and "h" = high. I shall convert the fire_M6 confidence column to a qualitative column as well

In [87]:
confidence_values = fire_V1['confidence'].unique()
confidence_numbers = fire_V1['confidence'].value_counts()

In [106]:
size = fire_V1['confidence'].size

In [107]:
size

1353795

In [92]:
for item in confidence_numbers:
    print(item)

1109821
156678
87296


for fire_V1 there are:
82% observations with normal confidence,
12% observations with low confidence,
 6% observations with high confidence.
I shall use these percentages for transforming fire_M6['confidence'] to a categorical feature

In [86]:
for item in confidence_numbers:
    print(item/fire_V1['confidence'].size)

0.8197851225628696
0.11573244102689108
0.06448243641023936


In [120]:
confidence_low_normal = [confidence_numbers[1]/size, (confidence_numbers[1]+confidence_numbers[0])/size]
print(confidence_low_normal[0], confidence_low_normal[1])
low_confidence = fire_M6['confidence'].quantile(confidence_low_normal[0])
medium_confidence = fire_M6['confidence'].quantile(confidence_low_normal[1])
print('Low confidence: numeric value < ', low_confidence,
      '\nMedium confidence: numeric value < ', medium_confidence,
      '\nHigh confidence: any other value')

0.11573244102689108 0.9355175635897607
Low confidence: numeric value <  41.0 
Medium confidence: numeric value <  99.0 
High confidence: any other value


In [165]:
fire_M6['low_confidence'] = pd.Series(np.NaN, index=fire_M6.index)
fire_M6['medium_confidence'] = pd.Series(np.NaN, index=fire_M6.index)
fire_M6['high_confidence'] = pd.Series(np.NaN, index=fire_M6.index)

In [166]:
fire_M6.loc[fire_M6['confidence'] < low_confidence, 'low_confidence'] = True
fire_M6.loc[fire_M6['confidence'] >= low_confidence, 'low_confidence'] = False

In [167]:
fire_M6.loc[fire_M6['confidence'] >= medium_confidence, 'high_confidence'] = True
fire_M6.loc[fire_M6['confidence'] < medium_confidence, 'high_confidence'] = False

In [168]:
fire_M6['medium_confidence'] = ~(fire_M6['low_confidence'] | fire_M6['high_confidence'])

In [171]:
fire_M6.drop('confidence', axis=1, inplace=True)

In [184]:
fire_M6.head()

,acq_date,frp,latitude,longitude,decimal_time,mean_brightness,low_confidence,medium_confidence,high_confidence
0,2000-11-01,26.8,-20.8650,146.3039,0.01181,317.00,True,False,False
1,2000-11-01,68.8,-20.5602,146.0194,0.01181,319.95,False,True,False
2,2000-11-01,7.1,-24.3197,151.5619,0.01250,311.15,False,True,False
3,2000-11-01,10.5,-18.0912,128.9856,0.07986,327.20,False,True,False
4,2000-11-01,100.3,-25.5102,125.4199,0.08125,342.75,False,False,True


In [173]:
fire_V1['low_confidence'] = pd.Series(np.NaN, index=fire_V1.index)
fire_V1['medium_confidence'] = pd.Series(np.NaN, index=fire_V1.index)
fire_V1['high_confidence'] = pd.Series(np.NaN, index=fire_V1.index)

In [180]:
fire_V1['low_confidence'] = (fire_V1['confidence']=='l')
fire_V1['medium_confidence'] = (fire_V1['confidence']=='n')
fire_V1['high_confidence'] = (fire_V1['confidence']=='h')

In [182]:
fire_V1.drop('confidence', axis=1, inplace=True)

In [183]:
fire_V1.head()

,acq_date,frp,latitude,longitude,decimal_time,mean_brightness,low_confidence,medium_confidence,high_confidence
6,2012-01-31,0.9,11.81456,4.97983,0.05208,298.45,False,True,False
7,2012-01-31,1.3,10.39675,13.45527,0.05208,302.25,False,True,False
8,2012-01-31,1.9,13.73985,13.83765,0.05208,301.80,False,True,False
9,2012-01-31,1.1,10.61242,13.48132,0.05208,298.25,False,True,False
10,2012-01-31,1.8,9.41623,-2.80546,0.05278,299.55,False,True,False


In [283]:
# Save fire_M6 dataframe to a file, load it and compare it with the saved dataframe
fire_M6.to_csv('./Data/fire/fire_M6_final.csv',index=False)
fire_M6_new_df=pd.read_csv('./Data/fire/fire_M6_final.csv', index_col = False)

In [285]:
fire_M6.head()

,acq_date,frp,latitude,longitude,decimal_time,mean_brightness,low_confidence,medium_confidence,high_confidence
0,2000-11-01,26.8,-20.8650,146.3039,0.01181,317.00,True,False,False
1,2000-11-01,68.8,-20.5602,146.0194,0.01181,319.95,False,True,False
2,2000-11-01,7.1,-24.3197,151.5619,0.01250,311.15,False,True,False
3,2000-11-01,10.5,-18.0912,128.9856,0.07986,327.20,False,True,False
4,2000-11-01,100.3,-25.5102,125.4199,0.08125,342.75,False,False,True


In [286]:
fire_M6_new_df.head()

,acq_date,frp,latitude,longitude,decimal_time,mean_brightness,low_confidence,medium_confidence,high_confidence
0,2000-11-01,26.8,-20.8650,146.3039,0.01181,317.00,True,False,False
1,2000-11-01,68.8,-20.5602,146.0194,0.01181,319.95,False,True,False
2,2000-11-01,7.1,-24.3197,151.5619,0.01250,311.15,False,True,False
3,2000-11-01,10.5,-18.0912,128.9856,0.07986,327.20,False,True,False
4,2000-11-01,100.3,-25.5102,125.4199,0.08125,342.75,False,False,True


In [287]:
fire_M6.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 817400 entries, 0 to 36736
Data columns (total 9 columns):
acq_date             817400 non-null object
frp                  817400 non-null float64
latitude             817400 non-null float64
longitude            817400 non-null float64
decimal_time         817400 non-null float64
mean_brightness      817400 non-null float64
low_confidence       817400 non-null bool
medium_confidence    817400 non-null bool
high_confidence      817400 non-null bool
dtypes: bool(3), float64(5), object(1)
memory usage: 66.0+ MB


In [288]:
fire_M6_new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 817400 entries, 0 to 817399
Data columns (total 9 columns):
acq_date             817400 non-null object
frp                  817400 non-null float64
latitude             817400 non-null float64
longitude            817400 non-null float64
decimal_time         817400 non-null float64
mean_brightness      817400 non-null float64
low_confidence       817400 non-null bool
medium_confidence    817400 non-null bool
high_confidence      817400 non-null bool
dtypes: bool(3), float64(5), object(1)
memory usage: 39.8+ MB


In [289]:
# Save fire_V1 dataframe to a file, load it and compare it with the saved dataframe
fire_V1.to_csv('./Data/fire/fire_V1_final.csv',index=False)
fire_V1_new_df=pd.read_csv('./Data/fire/fire_V1_final.csv', index_col = False)

In [290]:
fire_V1.head()

,acq_date,frp,latitude,longitude,decimal_time,mean_brightness,low_confidence,medium_confidence,high_confidence
6,2012-01-31,0.9,11.81456,4.97983,0.05208,298.45,False,True,False
7,2012-01-31,1.3,10.39675,13.45527,0.05208,302.25,False,True,False
8,2012-01-31,1.9,13.73985,13.83765,0.05208,301.80,False,True,False
9,2012-01-31,1.1,10.61242,13.48132,0.05208,298.25,False,True,False
10,2012-01-31,1.8,9.41623,-2.80546,0.05278,299.55,False,True,False


In [291]:
fire_V1_new_df.head()

,acq_date,frp,latitude,longitude,decimal_time,mean_brightness,low_confidence,medium_confidence,high_confidence
0,2012-01-31,0.9,11.81456,4.97983,0.05208,298.45,False,True,False
1,2012-01-31,1.3,10.39675,13.45527,0.05208,302.25,False,True,False
2,2012-01-31,1.9,13.73985,13.83765,0.05208,301.80,False,True,False
3,2012-01-31,1.1,10.61242,13.48132,0.05208,298.25,False,True,False
4,2012-01-31,1.8,9.41623,-2.80546,0.05278,299.55,False,True,False


In [292]:
fire_V1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1353795 entries, 6 to 87314
Data columns (total 9 columns):
acq_date             1353795 non-null object
frp                  1353795 non-null float64
latitude             1353795 non-null float64
longitude            1353795 non-null float64
decimal_time         1353795 non-null float64
mean_brightness      1353795 non-null float64
low_confidence       1353795 non-null bool
medium_confidence    1353795 non-null bool
high_confidence      1353795 non-null bool
dtypes: bool(3), float64(5), object(1)
memory usage: 76.2+ MB


In [293]:
fire_V1_new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1353795 entries, 0 to 1353794
Data columns (total 9 columns):
acq_date             1353795 non-null object
frp                  1353795 non-null float64
latitude             1353795 non-null float64
longitude            1353795 non-null float64
decimal_time         1353795 non-null float64
mean_brightness      1353795 non-null float64
low_confidence       1353795 non-null bool
medium_confidence    1353795 non-null bool
high_confidence      1353795 non-null bool
dtypes: bool(3), float64(5), object(1)
memory usage: 65.8+ MB


CO2

In [185]:
ls -ltrh ./Data/co2

total 168
-rw-r--r--@ 1 bogdan  staff    48K Nov 11 21:44 monthly_in_situ_co2.csv
-rw-r--r--@ 1 bogdan  staff    29K Nov 21 18:58 co2_mm_mlo.csv


In [186]:
co2_df = pd.read_csv('./Data/co2/co2_mm_mlo.csv')

In [187]:
co2_df.head()

,Year,Month,decimal_date,average,interpolated,trend,days_season_corr
0,1958,3,1958.208,315.71,315.71,314.62,-1
1,1958,4,1958.292,317.45,317.45,315.29,-1
2,1958,5,1958.375,317.50,317.50,314.71,-1
3,1958,6,1958.458,-99.99,317.10,314.85,-1
4,1958,7,1958.542,315.86,315.86,314.98,-1


Drop useless columns

In [188]:
co2_df.drop(['average', 'days_season_corr'], axis=1, inplace=True)

In [220]:
co2_df = co2_df.rename(columns={"interpolated": "CO2_ppm"})

In [221]:
co2_df.head()

,Year,Month,decimal_date,CO2_ppm,trend
0,1958,3,1958.208,315.71,314.62
1,1958,4,1958.292,317.45,315.29
2,1958,5,1958.375,317.50,314.71
3,1958,6,1958.458,317.10,314.85
4,1958,7,1958.542,315.86,314.98


In [222]:
co2_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 740 entries, 0 to 739
Data columns (total 5 columns):
Year            740 non-null int64
Month           740 non-null int64
decimal_date    740 non-null float64
CO2_ppm         740 non-null float64
trend           740 non-null float64
dtypes: float64(3), int64(2)
memory usage: 29.0 KB


In [279]:
co2_df.to_csv('./Data/CO2/CO2_final.csv',index=False)
new_df=pd.read_csv('./Data/CO2/CO2_final.csv', index_col = False)

In [282]:
new_df.head()

,Year,Month,decimal_date,CO2_ppm,trend
0,1958,3,1958.208,315.71,314.62
1,1958,4,1958.292,317.45,315.29
2,1958,5,1958.375,317.50,314.71
3,1958,6,1958.458,317.10,314.85
4,1958,7,1958.542,315.86,314.98


ICE

In [294]:
ls -ltrh Data/ice/*.csv

-rw-rw-rw-  1 bogdan  staff   2.2M Oct 24 11:58 Data/ice/N_seaice_extent_daily_v3.0.csv
-rw-rw-rw-  1 bogdan  staff   2.2M Oct 24 12:03 Data/ice/S_seaice_extent_daily_v3.0.csv


In [295]:
less ./Data/ice/N_seaice_extent_daily_v3.0.csv

In [296]:
less ./Data/ice/S_seaice_extent_daily_v3.0.csv

In [297]:
# Read the two files
ice_north_df = pd.read_csv('Data/ice/N_seaice_extent_daily_v3.0.csv')
ice_south_df = pd.read_csv('Data/ice/S_seaice_extent_daily_v3.0.csv')

In [298]:
ice_north_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13322 entries, 0 to 13321
Data columns (total 6 columns):
Year            13322 non-null object
 Month          13322 non-null object
 Day            13322 non-null object
     Extent     13322 non-null object
    Missing     13322 non-null object
 Source Data    13322 non-null object
dtypes: object(6)
memory usage: 624.5+ KB


In [299]:
ice_north_df.head()

,Year,Month,Day,Extent,Missing,Source Data
0,YYYY,MM,DD,10^6 sq km,10^6 sq km,Source data product web sites: http://nsidc.o...
1,1978,10,26,10.231,0.000,['ftp://sidads.colorado.edu/pub/DATASETS/nsid...
2,1978,10,28,10.420,0.000,['ftp://sidads.colorado.edu/pub/DATASETS/nsid...
3,1978,10,30,10.557,0.000,['ftp://sidads.colorado.edu/pub/DATASETS/nsid...
4,1978,11,01,10.670,0.000,['ftp://sidads.colorado.edu/pub/DATASETS/nsid...


In [300]:
ice_north_df.columns

Index(['Year', ' Month', ' Day', '     Extent', '    Missing', ' Source Data'], dtype='object')

In [301]:
ice_south_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13322 entries, 0 to 13321
Data columns (total 6 columns):
Year            13322 non-null object
 Month          13322 non-null object
 Day            13322 non-null object
     Extent     13322 non-null object
    Missing     13322 non-null object
 Source Data    13322 non-null object
dtypes: object(6)
memory usage: 624.5+ KB


In [302]:
ice_south_df.head()

,Year,Month,Day,Extent,Missing,Source Data
0,YYYY,MM,DD,10^6 sq km,10^6 sq km,Source data product web sites: http://nsidc.o...
1,1978,10,26,17.624,0.000,['ftp://sidads.colorado.edu/pub/DATASETS/nsid...
2,1978,10,28,17.803,0.000,['ftp://sidads.colorado.edu/pub/DATASETS/nsid...
3,1978,10,30,17.670,0.000,['ftp://sidads.colorado.edu/pub/DATASETS/nsid...
4,1978,11,01,17.527,0.000,['ftp://sidads.colorado.edu/pub/DATASETS/nsid...


In [303]:
ice_south_df.columns

Index(['Year', ' Month', ' Day', '     Extent', '    Missing', ' Source Data'], dtype='object')

In [304]:
# There are a few records for 'S_seaice_extent_daily_v3.0.csv' with missing areas but these are less than 0.001
# These missing values will just be ignored
ice_south_df[1::]['    Missing'].astype(float)[ice_south_df[1::]['    Missing'].astype(float)!=0.]

13040    0.001
13188    0.001
13214    0.001
13302    0.002
13310    0.001
Name:     Missing, dtype: float64

In [305]:
# No missing areas for 'S_seaice_extent_daily_v3.0.csv'
ice_north_df[1::]['    Missing'].astype(float)[ice_north_df[1::]['    Missing'].astype(float)!=0.]

Series([], Name:     Missing, dtype: float64)

In [306]:
# drop useless columns
ice_north_df.drop(['    Missing', ' Source Data'], axis=1, inplace=True)
ice_south_df.drop(['    Missing', ' Source Data'], axis=1, inplace=True)

In [307]:
#Create a date column for N and S, wth the format 'YYYY-MM-DD'
ice_north_df['acq_date'] = ice_north_df.iloc[:,0].str.strip() + \
'-' + ice_north_df.iloc[:,1].str.strip() + \
'-' + ice_north_df.iloc[:,2].str.strip()
ice_south_df['acq_date'] = ice_south_df.iloc[:,0].str.strip() + \
'-' + ice_south_df.iloc[:,1].str.strip() + \
'-' + ice_south_df.iloc[:,2].str.strip()

In [308]:
ice_north_df.head()

,Year,Month,Day,Extent,acq_date
0,YYYY,MM,DD,10^6 sq km,YYYY-MM-DD
1,1978,10,26,10.231,1978-10-26
2,1978,10,28,10.420,1978-10-28
3,1978,10,30,10.557,1978-10-30
4,1978,11,01,10.670,1978-11-01


In [309]:
# Drop first line and the line for 1978-10-26 (there is no regional data in the Excel file for that date)
ice_north_df.drop(ice_north_df.index[[0, 1]], inplace=True)

In [310]:
ice_north_df.head()

,Year,Month,Day,Extent,acq_date
2,1978,10,28,10.420,1978-10-28
3,1978,10,30,10.557,1978-10-30
4,1978,11,01,10.670,1978-11-01
5,1978,11,03,10.777,1978-11-03
6,1978,11,05,10.968,1978-11-05


In [311]:
ice_south_df.head()

,Year,Month,Day,Extent,acq_date
0,YYYY,MM,DD,10^6 sq km,YYYY-MM-DD
1,1978,10,26,17.624,1978-10-26
2,1978,10,28,17.803,1978-10-28
3,1978,10,30,17.670,1978-10-30
4,1978,11,01,17.527,1978-11-01


In [312]:
ice_south_df.drop(ice_south_df.index[[0, 1]], inplace=True)

In [313]:
ice_south_df.head()

,Year,Month,Day,Extent,acq_date
2,1978,10,28,17.803,1978-10-28
3,1978,10,30,17.670,1978-10-30
4,1978,11,01,17.527,1978-11-01
5,1978,11,03,17.486,1978-11-03
6,1978,11,05,17.343,1978-11-05


In [314]:
# create different column names for the two files
ice_north_df['extent_north'] = ice_north_df.iloc[:,3].str.strip().astype(float)
ice_south_df['extent_south'] = ice_south_df.iloc[:,3].str.strip().astype(float)

In [315]:
# Drop useless columns
ice_north_df = ice_north_df.drop(ice_north_df.columns[[0, 1, 2, 3]], axis=1)
ice_south_df = ice_south_df.drop(ice_south_df.columns[[0, 1, 2, 3]], axis=1)

In [316]:
# Set the same index for the two dataframes
ice_north_df.set_index('acq_date', inplace=True)
ice_south_df.set_index('acq_date', inplace=True)

In [317]:
# Concatenate the two dataframes
ice_df = pd.concat([ice_north_df, ice_south_df], axis=1)

In [318]:
ice_df

,extent_north,extent_south
acq_date,,
1978-10-28,10.420,17.803
1978-10-30,10.557,17.670
1978-11-01,10.670,17.527
1978-11-03,10.777,17.486
1978-11-05,10.968,17.343
1978-11-07,11.080,17.157
1978-11-09,11.189,17.028
1978-11-11,11.314,16.815
1978-11-13,11.460,16.371


In [319]:
ice_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13320 entries, 1978-10-28 to 2019-10-22
Data columns (total 2 columns):
extent_north    13320 non-null float64
extent_south    13320 non-null float64
dtypes: float64(2)
memory usage: 312.2+ KB


In [320]:
ice_df.describe()

,extent_north,extent_south
count,13320.000000,13320.000000
mean,11.312141,11.677272
std,3.288076,5.640254
min,3.340000,2.080000
25%,8.572750,6.071750
50%,12.010000,12.632500
75%,14.220000,17.240250
max,16.635000,20.201000


In [321]:
# Check for null values
ice_df.isnull().sum().sum()

0

ICE Regional

In [322]:
ls -ltrh Data/ice/Sea_Ice_Index_Regional_Daily_Data_G02135_v3.0.xlsx

-rw-rw-rw-@ 1 bogdan  staff   3.2M Oct 24 12:05 Data/ice/Sea_Ice_Index_Regional_Daily_Data_G02135_v3.0.xlsx


In [323]:
# Read Ice Regional file
ice_regional_df = pd.ExcelFile('Data/ice/Sea_Ice_Index_Regional_Daily_Data_G02135_v3.0.xlsx')
ice_regional_df.sheet_names

['Baffin-Area-km^2',
 'Baffin-Extent-km^2',
 'Barents-Area-km^2',
 'Barents-Extent-km^2',
 'Beaufort-Area-km^2',
 'Beaufort-Extent-km^2',
 'Bering-Area-km^2',
 'Bering-Extent-km^2',
 'CanadianArchipelago-Area-km^2',
 'CanadianArchipelago-Extent-km^2',
 'Central-Arctic-Area-km^2',
 'Central-Arctic-Extent-km^2',
 'Chukchi-Area-km^2',
 'Chukchi-Extent-km^2',
 'East-Siberian-Area-km^2',
 'East-Siberian-Extent-km^2',
 'Greenland-Area-km^2',
 'Greenland-Extent-km^2',
 'Hudson-Area-km^2',
 'Hudson-Extent-km^2',
 'Kara-Area-km^2',
 'Kara-Extent-km^2',
 'Laptev-Area-km^2',
 'Laptev-Extent-km^2',
 'Okhotsk-Area-km^2',
 'Okhotsk-Extent-km^2',
 'St-Lawrence-Area-km^2',
 'St-Lawrence-Extent-km^2',
 'Documentation']

In [324]:
# Check the structure of the excel file
# The month name is written only once, for the first day on each month
test_df=ice_regional_df.parse('Baffin-Extent-km^2')
test_df.head(35)

,month,day,1978,1979,1980,1981,1982,1983,1984,1985,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,January,1,NaN,903186.035,1013769.883,1033270.716,1023094.590,1347723.970,1309901.559,1341199.586,...,795608.107,657280.210,970785.241,836904.594,1042165.178,919851.203,1107737.558,1043957.619,852411.168,917527.054
1,NaN,2,NaN,928997.908,999873.492,1050801.408,1021066.245,1372848.377,1350548.340,1360997.397,...,790348.007,665528.563,988757.101,846580.138,1045929.234,952044.039,1118000.089,1041095.765,872204.561,942891.608
2,NaN,3,NaN,947432.787,1000897.098,1041880.956,1013746.335,1417371.453,1372754.604,1376789.972,...,786922.827,670551.044,999673.091,853340.629,1044428.577,984973.384,1126274.372,1039445.720,891754.362,961108.705
3,NaN,4,NaN,974928.335,997687.620,1037514.096,1030366.773,1460409.917,1392297.001,1398633.692,...,784717.294,670551.024,1006333.892,863309.928,1047832.206,1008812.804,1131821.502,1040792.446,912064.230,985959.419
4,NaN,5,NaN,1005270.542,1015031.822,1018820.331,1040308.602,1491475.904,1431611.850,1421457.076,...,787263.871,668051.543,1010444.124,877502.215,1047953.087,1031026.911,1137901.759,1055976.206,926768.633,1002583.776
5,NaN,6,NaN,1020866.519,1026332.269,1002736.428,1039142.313,1530012.742,1441697.272,1425325.650,...,783989.956,666964.802,1012726.638,896387.766,1045552.610,1041425.668,1143122.799,1073472.421,928374.126,1019351.792
6,NaN,7,NaN,1040988.952,1069400.637,1010485.255,1050208.692,1531359.065,1446624.956,1437691.966,...,784935.765,659703.793,1012993.471,918865.668,1051147.246,1051546.883,1146589.922,1077058.157,921234.302,1030276.454
7,NaN,8,NaN,1064087.127,1102628.800,1003707.856,1033289.669,1543829.795,1439200.445,1416160.241,...,792010.474,652593.179,1020406.891,937745.734,1054370.214,1067754.241,1147676.338,1081727.365,913151.686,1041308.047
8,NaN,9,NaN,1081170.976,1108456.284,1032386.264,1018130.679,1559868.703,1443109.770,1403079.796,...,800453.302,651912.342,1037356.960,961546.402,1054033.087,1090845.365,1147695.498,1082576.027,906364.321,1045206.417
9,NaN,10,NaN,1094338.710,1137824.314,1057862.996,1019872.450,1562999.115,1442072.753,1393712.912,...,811319.869,655219.873,1055413.543,979023.324,1063678.220,1107823.450,1154062.400,1087178.770,908123.416,1053270.988


In [325]:
# Ignore last tab ('Documentation')
regional_dict = dict()
for sheet in ice_regional_df.sheet_names[:-1]:
    regional_dict[sheet]=ice_regional_df.parse(sheet)

In [326]:
regional_dict.keys()

dict_keys(['Baffin-Area-km^2', 'Baffin-Extent-km^2', 'Barents-Area-km^2', 'Barents-Extent-km^2', 'Beaufort-Area-km^2', 'Beaufort-Extent-km^2', 'Bering-Area-km^2', 'Bering-Extent-km^2', 'CanadianArchipelago-Area-km^2', 'CanadianArchipelago-Extent-km^2', 'Central-Arctic-Area-km^2', 'Central-Arctic-Extent-km^2', 'Chukchi-Area-km^2', 'Chukchi-Extent-km^2', 'East-Siberian-Area-km^2', 'East-Siberian-Extent-km^2', 'Greenland-Area-km^2', 'Greenland-Extent-km^2', 'Hudson-Area-km^2', 'Hudson-Extent-km^2', 'Kara-Area-km^2', 'Kara-Extent-km^2', 'Laptev-Area-km^2', 'Laptev-Extent-km^2', 'Okhotsk-Area-km^2', 'Okhotsk-Extent-km^2', 'St-Lawrence-Area-km^2', 'St-Lawrence-Extent-km^2'])

In [327]:
regional_dict['Baffin-Area-km^2']

,month,day,1978,1979,1980,1981,1982,1983,1984,1985,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,January,1,NaN,677547.703,845683.988,831565.263,787287.378,1169342.437,1078749.266,1062856.098,...,595951.256,501285.997,791899.698,663457.962,837240.367,732812.444,881084.098,823487.089,702035.905,746880.532
1,NaN,2,NaN,700084.156,829620.808,854534.647,796243.810,1195146.784,1106676.585,1085802.451,...,590370.369,500635.821,811486.519,683095.271,836211.440,752802.009,902299.171,819038.661,721057.378,766064.811
2,NaN,3,NaN,720887.777,823613.861,852476.913,804236.368,1220321.638,1118567.884,1111163.813,...,590993.097,500511.602,827528.849,700058.956,836711.378,779433.092,923286.442,814502.357,743894.559,783298.934
3,NaN,4,NaN,735098.275,809752.946,853556.592,807509.957,1242555.366,1140125.320,1141988.695,...,595130.888,497089.592,844754.953,713693.618,838393.908,798901.939,937252.304,812677.855,767072.209,802206.398
4,NaN,5,NaN,754338.165,819375.022,841279.513,814106.904,1260648.001,1172885.708,1167666.807,...,603640.935,490688.361,860541.372,726486.039,833940.445,814070.564,943778.075,832308.872,782108.664,817302.434
5,NaN,6,NaN,761438.929,825109.571,832543.399,819019.984,1283752.309,1182918.366,1165526.650,...,611249.303,491809.787,874965.137,744403.963,833732.779,822909.105,944675.470,845652.740,783014.836,830871.334
6,NaN,7,NaN,769579.863,833335.356,830258.013,821451.639,1299176.690,1193111.938,1182442.395,...,621746.352,494168.916,884056.986,762800.296,846030.160,834487.056,943114.809,849642.994,775116.723,836828.283
7,NaN,8,NaN,796852.863,844203.052,821206.297,805522.879,1313429.363,1197284.571,1153923.236,...,634884.534,494401.636,893923.846,771992.702,843646.026,849002.579,939991.847,856784.530,762819.840,841274.989
8,NaN,9,NaN,813519.659,848927.900,835694.143,801255.751,1331548.773,1204306.760,1129065.625,...,649274.206,498482.704,906288.033,782291.116,837294.071,870870.362,941746.052,867481.069,748945.585,842730.257
9,NaN,10,NaN,838432.098,854082.263,845178.539,789880.919,1348906.524,1220694.716,1112886.199,...,665990.370,507214.906,915939.838,789849.222,836797.909,892109.836,953905.574,875138.915,742212.875,849309.718


In [328]:
# There are NaN values that must be taken care of
regional_dict['Baffin-Area-km^2'].loc[:, ['month', 'day', 2019]][regional_dict['Baffin-Area-km^2'].loc[:, 2019].isnull()]

,month,day,2019
59,NaN,29,NaN
296,NaN,23,NaN
297,NaN,24,NaN
298,NaN,25,NaN
299,NaN,26,NaN
300,NaN,27,NaN
301,NaN,28,NaN
302,NaN,29,NaN
303,NaN,30,NaN
304,NaN,31,NaN


In [329]:
# Melt the years from the dataframe
regional_updated_dict = dict()
for key in regional_dict.keys():
    regional_updated_dict.update({key:pd.melt(regional_dict[key], id_vars=['month', 'day'], var_name='year')})

In [330]:
regional_updated_dict['Baffin-Area-km^2']

,month,day,year,value
0,January,1,1978,NaN
1,NaN,2,1978,NaN
2,NaN,3,1978,NaN
3,NaN,4,1978,NaN
4,NaN,5,1978,NaN
5,NaN,6,1978,NaN
6,NaN,7,1978,NaN
7,NaN,8,1978,NaN
8,NaN,9,1978,NaN
9,NaN,10,1978,NaN


In [331]:
# forward fill the month for all tabs
for key in regional_dict.keys():
    regional_updated_dict[key]['month'] = regional_updated_dict[key]['month'].ffill()

In [332]:
regional_updated_dict['Baffin-Area-km^2']

,month,day,year,value
0,January,1,1978,NaN
1,January,2,1978,NaN
2,January,3,1978,NaN
3,January,4,1978,NaN
4,January,5,1978,NaN
5,January,6,1978,NaN
6,January,7,1978,NaN
7,January,8,1978,NaN
8,January,9,1978,NaN
9,January,10,1978,NaN


In [191]:
regional_updated_dict['Baffin-Extent-km^2']

,month,day,year,value
0,January,1,1978,NaN
1,January,2,1978,NaN
2,January,3,1978,NaN
3,January,4,1978,NaN
4,January,5,1978,NaN
5,January,6,1978,NaN
6,January,7,1978,NaN
7,January,8,1978,NaN
8,January,9,1978,NaN
9,January,10,1978,NaN


In [333]:
def two_digits_month(Month):
    return {
        'January' : '01',
        'February' : '02',
        'March' : '03',
        'April' : '04',
        'May' : '05',
        'June' : '06',
        'July' : '07',
        'August' : '08',
        'September' : '09', 
        'October' : '10',
        'November' : '11',
        'December' : '12'
}[Month]

In [334]:
# Check the month conversion
two_digits_month('March')

'03'

In [335]:
for key in regional_dict.keys():
    acq_date_list = []
    for item in range(len(regional_updated_dict[key])):
        acq_date = str(regional_updated_dict[key]['year'][item]) + '-' + \
        two_digits_month(regional_updated_dict[key]['month'][item]) + '-' + \
        str(regional_updated_dict[key]['day'][item]).zfill(2)
        acq_date_list.append(acq_date)
    regional_updated_dict[key]['acq_date'] = acq_date_list

In [336]:
regional_updated_dict['Baffin-Area-km^2']

,month,day,year,value,acq_date
0,January,1,1978,NaN,1978-01-01
1,January,2,1978,NaN,1978-01-02
2,January,3,1978,NaN,1978-01-03
3,January,4,1978,NaN,1978-01-04
4,January,5,1978,NaN,1978-01-05
5,January,6,1978,NaN,1978-01-06
6,January,7,1978,NaN,1978-01-07
7,January,8,1978,NaN,1978-01-08
8,January,9,1978,NaN,1978-01-09
9,January,10,1978,NaN,1978-01-10


In [337]:
regional_updated_dict['Baffin-Extent-km^2']

,month,day,year,value,acq_date
0,January,1,1978,NaN,1978-01-01
1,January,2,1978,NaN,1978-01-02
2,January,3,1978,NaN,1978-01-03
3,January,4,1978,NaN,1978-01-04
4,January,5,1978,NaN,1978-01-05
5,January,6,1978,NaN,1978-01-06
6,January,7,1978,NaN,1978-01-07
7,January,8,1978,NaN,1978-01-08
8,January,9,1978,NaN,1978-01-09
9,January,10,1978,NaN,1978-01-10


In [338]:
for key in regional_dict.keys():
    regional_updated_dict[key].drop(['month', 'day', 'year'], axis=1, inplace=True)

In [339]:
regional_updated_dict['Baffin-Area-km^2'].columns

Index(['value', 'acq_date'], dtype='object')

In [340]:
regional_updated_dict['Baffin-Extent-km^2'].columns

Index(['value', 'acq_date'], dtype='object')

In [341]:
# Set acq_date as the index and excel tab name as the column name
for key in regional_dict.keys():
    regional_updated_dict[key].set_index('acq_date', inplace=True)
    regional_updated_dict[key].columns = [key]

In [342]:
regional_updated_dict['Baffin-Area-km^2']

,Baffin-Area-km^2
acq_date,
1978-01-01,NaN
1978-01-02,NaN
1978-01-03,NaN
1978-01-04,NaN
1978-01-05,NaN
1978-01-06,NaN
1978-01-07,NaN
1978-01-08,NaN
1978-01-09,NaN


In [343]:
regional_updated_dict['Baffin-Extent-km^2']

,Baffin-Extent-km^2
acq_date,
1978-01-01,NaN
1978-01-02,NaN
1978-01-03,NaN
1978-01-04,NaN
1978-01-05,NaN
1978-01-06,NaN
1978-01-07,NaN
1978-01-08,NaN
1978-01-09,NaN


In [344]:
# There are 3 reasons for null values:
# - the date is before the start of recorded data
# - the date is in 2019, after the moment when the data was gathered
# - 29 february has a not null value only for leap years
# Therefore, all these observations with NaN values can be dropped
regional_updated_dict['Baffin-Area-km^2'][regional_updated_dict['Baffin-Area-km^2'].isnull()]

,Baffin-Area-km^2
acq_date,
1978-01-01,NaN
1978-01-02,NaN
1978-01-03,NaN
1978-01-04,NaN
1978-01-05,NaN
1978-01-06,NaN
1978-01-07,NaN
1978-01-08,NaN
1978-01-09,NaN


In [345]:
for key in regional_dict.keys():
    regional_updated_dict[key] = regional_updated_dict[key].dropna()

In [346]:
# These are the final time series for regional data:
# 1978-10-28 (start of data gathering) to 2019-10-22 (the date when the excel file was created)
regional_updated_dict['Baffin-Area-km^2']

,Baffin-Area-km^2
acq_date,
1978-10-28,416908.720
1978-10-29,416908.720
1978-10-30,419829.102
1978-10-31,424603.699
1978-11-01,432456.540
1978-11-02,436916.044
1978-11-03,453981.625
1978-11-04,468137.504
1978-11-05,478228.269


In [347]:
regional_updated_dict['Baffin-Extent-km^2']

,Baffin-Extent-km^2
acq_date,
1978-10-28,521862.289
1978-10-29,521862.289
1978-10-30,531533.703
1978-10-31,539494.751
1978-11-01,549767.557
1978-11-02,560594.850
1978-11-03,572481.084
1978-11-04,583283.360
1978-11-05,594298.364


In [348]:
regional_updated_dict['Baffin-Area-km^2'].isnull().sum().sum()

0

In [349]:
regional_updated_dict['Baffin-Extent-km^2'].isnull().sum().sum()

0

In [350]:
final_ice_df = pd.DataFrame()
for key in regional_dict.keys():
    final_ice_df = pd.concat([final_ice_df, regional_updated_dict[key]], axis=1, sort=False)

In [351]:
final_ice_df

,Baffin-Area-km^2,Baffin-Extent-km^2,Barents-Area-km^2,Barents-Extent-km^2,Beaufort-Area-km^2,Beaufort-Extent-km^2,Bering-Area-km^2,Bering-Extent-km^2,CanadianArchipelago-Area-km^2,CanadianArchipelago-Extent-km^2,...,Hudson-Area-km^2,Hudson-Extent-km^2,Kara-Area-km^2,Kara-Extent-km^2,Laptev-Area-km^2,Laptev-Extent-km^2,Okhotsk-Area-km^2,Okhotsk-Extent-km^2,St-Lawrence-Area-km^2,St-Lawrence-Extent-km^2
acq_date,,,,,,,,,,,,,,,,,,,,,
1978-10-28,416908.720,521862.289,222781.361,361743.416,895310.805,940219.390,11472.805,35555.883,719220.320,761228.252,...,246180.927,378913.635,569610.622,815398.750,802884.363,872414.226,16799.581,35135.582,0.000,0.000
1978-10-29,416908.720,521862.289,222781.361,361743.416,895310.805,940219.390,11472.805,35555.883,719220.320,761228.252,...,246180.927,378913.635,569610.622,815398.750,802884.363,872414.226,16799.581,35135.582,0.000,0.000
1978-10-30,419829.102,531533.703,229769.256,379395.172,894088.098,940219.390,13553.099,42670.305,717303.629,761228.252,...,261666.759,395635.784,562328.391,816017.367,806760.991,872414.226,18745.308,39102.698,0.000,0.000
1978-10-31,424603.699,539494.751,239080.138,397831.010,894437.562,940219.390,17155.632,54445.683,719259.986,761228.252,...,274520.725,411278.220,563103.427,820197.687,803822.189,872414.226,19828.268,41864.345,0.000,0.000
1978-11-01,432456.540,549767.557,255871.199,407009.512,891797.069,940219.390,16204.616,51087.306,713567.945,761228.252,...,282447.249,421828.771,592522.327,834384.662,801273.832,872414.226,19406.867,44005.995,967.403,2028.863
1978-11-02,436916.044,560594.850,266599.184,420032.599,889079.383,940219.390,16008.135,50634.850,707827.056,761228.252,...,295469.361,436004.978,599562.028,840006.606,805345.682,872414.226,20600.413,47663.113,1451.104,3043.295
1978-11-03,453981.625,572481.084,273238.066,428925.766,886904.020,940219.390,15146.336,45128.824,707977.925,761228.252,...,296513.547,441863.442,631868.299,852835.709,804449.229,872414.226,23336.706,55765.457,2106.390,3719.596
1978-11-04,468137.504,583283.360,287984.576,436039.307,884534.688,940219.390,13862.659,39243.663,705231.764,761228.252,...,298451.109,448255.122,673920.484,870626.263,799416.721,872414.226,23686.678,60129.719,3159.586,5579.395
1978-11-05,478228.269,594298.364,301386.462,446784.785,878757.693,940219.390,15285.012,46435.094,705065.476,761228.252,...,308529.466,464586.785,698110.232,877851.959,803279.392,872414.226,25970.600,62922.233,3180.264,5248.084


In [352]:
for key in regional_updated_dict.keys():
    print(key)

Baffin-Area-km^2
Baffin-Extent-km^2
Barents-Area-km^2
Barents-Extent-km^2
Beaufort-Area-km^2
Beaufort-Extent-km^2
Bering-Area-km^2
Bering-Extent-km^2
CanadianArchipelago-Area-km^2
CanadianArchipelago-Extent-km^2
Central-Arctic-Area-km^2
Central-Arctic-Extent-km^2
Chukchi-Area-km^2
Chukchi-Extent-km^2
East-Siberian-Area-km^2
East-Siberian-Extent-km^2
Greenland-Area-km^2
Greenland-Extent-km^2
Hudson-Area-km^2
Hudson-Extent-km^2
Kara-Area-km^2
Kara-Extent-km^2
Laptev-Area-km^2
Laptev-Extent-km^2
Okhotsk-Area-km^2
Okhotsk-Extent-km^2
St-Lawrence-Area-km^2
St-Lawrence-Extent-km^2


In [353]:
final_ice_df.columns

Index(['Baffin-Area-km^2', 'Baffin-Extent-km^2', 'Barents-Area-km^2',
       'Barents-Extent-km^2', 'Beaufort-Area-km^2', 'Beaufort-Extent-km^2',
       'Bering-Area-km^2', 'Bering-Extent-km^2',
       'CanadianArchipelago-Area-km^2', 'CanadianArchipelago-Extent-km^2',
       'Central-Arctic-Area-km^2', 'Central-Arctic-Extent-km^2',
       'Chukchi-Area-km^2', 'Chukchi-Extent-km^2', 'East-Siberian-Area-km^2',
       'East-Siberian-Extent-km^2', 'Greenland-Area-km^2',
       'Greenland-Extent-km^2', 'Hudson-Area-km^2', 'Hudson-Extent-km^2',
       'Kara-Area-km^2', 'Kara-Extent-km^2', 'Laptev-Area-km^2',
       'Laptev-Extent-km^2', 'Okhotsk-Area-km^2', 'Okhotsk-Extent-km^2',
       'St-Lawrence-Area-km^2', 'St-Lawrence-Extent-km^2'],
      dtype='object')

In [354]:
final_ice_df.isnull().sum().sum()

0

In [355]:
# In the first few years of observations, there are records for every second day.
# This issue must be resolved
ice_df

,extent_north,extent_south
acq_date,,
1978-10-28,10.420,17.803
1978-10-30,10.557,17.670
1978-11-01,10.670,17.527
1978-11-03,10.777,17.486
1978-11-05,10.968,17.343
1978-11-07,11.080,17.157
1978-11-09,11.189,17.028
1978-11-11,11.314,16.815
1978-11-13,11.460,16.371


In [356]:
final_ice_df = pd.concat([final_ice_df, ice_df], axis=1, sort=False)

In [362]:
final_ice_df.columns

Index(['Baffin-Area-km^2', 'Baffin-Extent-km^2', 'Barents-Area-km^2',
       'Barents-Extent-km^2', 'Beaufort-Area-km^2', 'Beaufort-Extent-km^2',
       'Bering-Area-km^2', 'Bering-Extent-km^2',
       'CanadianArchipelago-Area-km^2', 'CanadianArchipelago-Extent-km^2',
       'Central-Arctic-Area-km^2', 'Central-Arctic-Extent-km^2',
       'Chukchi-Area-km^2', 'Chukchi-Extent-km^2', 'East-Siberian-Area-km^2',
       'East-Siberian-Extent-km^2', 'Greenland-Area-km^2',
       'Greenland-Extent-km^2', 'Hudson-Area-km^2', 'Hudson-Extent-km^2',
       'Kara-Area-km^2', 'Kara-Extent-km^2', 'Laptev-Area-km^2',
       'Laptev-Extent-km^2', 'Okhotsk-Area-km^2', 'Okhotsk-Extent-km^2',
       'St-Lawrence-Area-km^2', 'St-Lawrence-Extent-km^2', 'extent_north',
       'extent_south'],
      dtype='object')

In [360]:
final_ice_df[['Baffin-Area-km^2', 'Baffin-Area-km^2', 'Barents-Area-km^2', 
              'Barents-Extent-km^2', 'extent_north', 'extent_south']].head()

,Baffin-Area-km^2,Baffin-Area-km^2,Barents-Area-km^2,Barents-Extent-km^2,extent_north,extent_south
1978-10-28,416908.720,416908.720,222781.361,361743.416,10.420,17.803
1978-10-29,416908.720,416908.720,222781.361,361743.416,NaN,NaN
1978-10-30,419829.102,419829.102,229769.256,379395.172,10.557,17.670
1978-10-31,424603.699,424603.699,239080.138,397831.010,NaN,NaN
1978-11-01,432456.540,432456.540,255871.199,407009.512,10.670,17.527


In [363]:
# As shown above, there are NaN values for the old 'extent_north' and 'extent_south' columns.
# The reason is that in the first few years sattelite data was available every second day
# We'll fill these NaN values by interpolating values from adjacent days
len(final_ice_df['extent_north'][final_ice_df['extent_north'].isnull()])

1612

In [364]:
len(final_ice_df['extent_south'][final_ice_df['extent_south'].isnull()])

1612

In [365]:
extent_north = final_ice_df['extent_north']
extent_south = final_ice_df['extent_south']

In [366]:
extent_north.head()

1978-10-28    10.420
1978-10-29       NaN
1978-10-30    10.557
1978-10-31       NaN
1978-11-01    10.670
Name: extent_north, dtype: float64

In [367]:
extent_south.head()

1978-10-28    17.803
1978-10-29       NaN
1978-10-30    17.670
1978-10-31       NaN
1978-11-01    17.527
Name: extent_south, dtype: float64

In [368]:
# Use an order 1 polynomial function for filling the missing data
# because there aren't huge differences in ice surface in 2 days
extent_north=extent_north.reset_index().interpolate(method='polynomial', order = 1)
extent_south=extent_south.reset_index().interpolate(method='polynomial', order = 1)

In [369]:
extent_north=extent_north.set_index('index')

In [370]:
extent_north.head()

,extent_north
index,
1978-10-28,10.4200
1978-10-29,10.4885
1978-10-30,10.5570
1978-10-31,10.6135
1978-11-01,10.6700


In [371]:
# The NaN at 1978-10-29 was filled correctly with the mean value from the previous and next observation

In [372]:
extent_south=extent_south.set_index('index')

In [373]:
extent_south.head()

,extent_south
index,
1978-10-28,17.8030
1978-10-29,17.7365
1978-10-30,17.6700
1978-10-31,17.5985
1978-11-01,17.5270


In [374]:
final_ice_df['extent_north'] = extent_north
final_ice_df['extent_south'] = extent_south

In [375]:
final_ice_df[['Baffin-Area-km^2', 'Baffin-Area-km^2', 'Barents-Area-km^2', 
              'Barents-Extent-km^2', 'extent_north', 'extent_south']].head()

,Baffin-Area-km^2,Baffin-Area-km^2,Barents-Area-km^2,Barents-Extent-km^2,extent_north,extent_south
1978-10-28,416908.720,416908.720,222781.361,361743.416,10.4200,17.8030
1978-10-29,416908.720,416908.720,222781.361,361743.416,10.4885,17.7365
1978-10-30,419829.102,419829.102,229769.256,379395.172,10.5570,17.6700
1978-10-31,424603.699,424603.699,239080.138,397831.010,10.6135,17.5985
1978-11-01,432456.540,432456.540,255871.199,407009.512,10.6700,17.5270


In [376]:
# There are no more nulls in the ice dataframe
len(final_ice_df['extent_north'][final_ice_df['extent_north'].isnull()])

0

In [409]:
final_ice_df.to_csv('./Data/ice/ice_final.csv',index=True)
final_ice_new_df=pd.read_csv('./Data/ice/ice_final.csv', index_col = False)
final_ice_new_df = final_ice_new_df.set_index('Unnamed: 0')
final_ice_new_df.index.name = None

In [410]:
final_ice_df.head(3)

,Baffin-Area-km^2,Baffin-Extent-km^2,Barents-Area-km^2,Barents-Extent-km^2,Beaufort-Area-km^2,Beaufort-Extent-km^2,Bering-Area-km^2,Bering-Extent-km^2,CanadianArchipelago-Area-km^2,CanadianArchipelago-Extent-km^2,...,Kara-Area-km^2,Kara-Extent-km^2,Laptev-Area-km^2,Laptev-Extent-km^2,Okhotsk-Area-km^2,Okhotsk-Extent-km^2,St-Lawrence-Area-km^2,St-Lawrence-Extent-km^2,extent_north,extent_south
1978-10-28,416908.720,521862.289,222781.361,361743.416,895310.805,940219.39,11472.805,35555.883,719220.320,761228.252,...,569610.622,815398.750,802884.363,872414.226,16799.581,35135.582,0.0,0.0,10.4200,17.8030
1978-10-29,416908.720,521862.289,222781.361,361743.416,895310.805,940219.39,11472.805,35555.883,719220.320,761228.252,...,569610.622,815398.750,802884.363,872414.226,16799.581,35135.582,0.0,0.0,10.4885,17.7365
1978-10-30,419829.102,531533.703,229769.256,379395.172,894088.098,940219.39,13553.099,42670.305,717303.629,761228.252,...,562328.391,816017.367,806760.991,872414.226,18745.308,39102.698,0.0,0.0,10.5570,17.6700


In [411]:
final_ice_new_df.head(3)

,Baffin-Area-km^2,Baffin-Extent-km^2,Barents-Area-km^2,Barents-Extent-km^2,Beaufort-Area-km^2,Beaufort-Extent-km^2,Bering-Area-km^2,Bering-Extent-km^2,CanadianArchipelago-Area-km^2,CanadianArchipelago-Extent-km^2,...,Kara-Area-km^2,Kara-Extent-km^2,Laptev-Area-km^2,Laptev-Extent-km^2,Okhotsk-Area-km^2,Okhotsk-Extent-km^2,St-Lawrence-Area-km^2,St-Lawrence-Extent-km^2,extent_north,extent_south
1978-10-28,416908.720,521862.289,222781.361,361743.416,895310.805,940219.39,11472.805,35555.883,719220.320,761228.252,...,569610.622,815398.750,802884.363,872414.226,16799.581,35135.582,0.0,0.0,10.4200,17.8030
1978-10-29,416908.720,521862.289,222781.361,361743.416,895310.805,940219.39,11472.805,35555.883,719220.320,761228.252,...,569610.622,815398.750,802884.363,872414.226,16799.581,35135.582,0.0,0.0,10.4885,17.7365
1978-10-30,419829.102,531533.703,229769.256,379395.172,894088.098,940219.39,13553.099,42670.305,717303.629,761228.252,...,562328.391,816017.367,806760.991,872414.226,18745.308,39102.698,0.0,0.0,10.5570,17.6700


In [2]:
!jupyter nbconvert --to html capstone-project1-data-wrangling.ipynb

[NbConvertApp] Converting notebook capstone-project1-data-wrangling.ipynb to html
[NbConvertApp] Writing 756365 bytes to capstone-project1-data-wrangling.html


In [3]:
!jupyter nbconvert --to pdf capstone-project1-data-wrangling.ipynb

[NbConvertApp] Converting notebook capstone-project1-data-wrangling.ipynb to pdf
[NbConvertApp] Writing 321160 bytes to ./notebook.tex
[NbConvertApp] Building PDF
Traceback (most recent call last):
  File "/Users/bogdan/anaconda3/bin/jupyter-nbconvert", line 11, in <module>
    sys.exit(main())
  File "/Users/bogdan/anaconda3/lib/python3.7/site-packages/jupyter_core/application.py", line 267, in launch_instance
    return super(JupyterApp, cls).launch_instance(argv=argv, **kwargs)
  File "/Users/bogdan/anaconda3/lib/python3.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/bogdan/anaconda3/lib/python3.7/site-packages/nbconvert/nbconvertapp.py", line 340, in start
    self.convert_notebooks()
  File "/Users/bogdan/anaconda3/lib/python3.7/site-packages/nbconvert/nbconvertapp.py", line 510, in convert_notebooks
    self.convert_single_notebook(notebook_filename)
  File "/Users/bogdan/anaconda3/lib/python3.7/site-packages/nbconver